# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [1]:
 !pip install "pymysql==1.0.2"

In [2]:
 !pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.8/2.1 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 9.8 MB/s eta 0:00:00


In [3]:
 !pip install mysql-connector-python

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.4 MB 9.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.4/16.4 MB 8.8 MB/s eta 0:00:02
   ------------- -------------------------- 5.5/16.4 MB 9.3 MB/s eta 0:00:02
   ------------------ --------------------- 7.6/16.4 MB 9.6 MB/s eta 0:00:01
   ------------------------ --------------- 10.2/16.4 MB 10.1 MB/s eta 0:00:01
   ------------------------------ --------- 12.6/16.4 MB 10.4 MB/s eta 0:00:01
   ------------------------------------- -- 15.2/16.4 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 10.4 MB/s eta 0:00:00


### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [4]:
host = "database-miercoles.cju6c4y488mh.eu-north-1.rds.amazonaws.com"
username = "admin"
password = "miercoles25+"
port = 3306

### Conexion BD

In [5]:
import pymysql

In [7]:
'''
pymysql.cursors.Dictcursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
                     )
# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [8]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta-
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries que modifiquen la base de datos de alguna manera:
de insert de datos, creación de tablas, etc 
'''

cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print (f"MysQL version: {version}")

MysQL version: {'VERSION()': '8.0.40'}


### Creación de DB

In [9]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [10]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [13]:
'''
Creamos una tabla 
Esto da error porque no le hemos dicho al servidor que BD
queremos usar
'''

# Crear una tabla que se llame country
create_table = '''
    CREATE TABLE country (
        id INT NOT NULL auto_increment,
        name TEXT,
        continent TEXT,
        population INT,
        gdp DOUBLE,
        primary key (id)
    )
'''
cursor.execute(create_table)

0

### Seleccionar la BD

In [12]:
# Para usar la BD recien creada
cursor.connection.commit()
use_db = '''USE country_database'''
cursor.execute(use_db)

0

In [14]:
# Chequear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_country_database': 'country'}]

In [ ]:
# Si queremos borrar alguna tabla 
# drop_table = '''DROP TABLE country'''TRUNCATE TABLE country elimina el contenido de la tabla
# y la deja vacia, pero no la elimina
# cursor.execute(drop_table)

### Insertar datos

In [15]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp)
VALUES ('Spain','Europe', 48, 1.62)
'''
cursor.execute(insert_data)

1

In [16]:
name = "Italy"
continent = "Europe"
population = 59
gdp = 2.3

In [17]:
insert_data = f'''
INSERT INTO country(name,continent,population,gdp) VALUES ("{name}","{continent}",{population},{gdp})
'''

In [18]:
cursor.execute(insert_data)

1

### Leer datos

In [19]:
query = '''
SELECT * FROM country WHERE name LIKE "%pa%"
'''
cursor.execute(query)

1

In [20]:
print(cursor.fetchall())

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 48, 'gdp': 1.62}]


### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [21]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [23]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 48, 'gdp': 1.62}, {'id': 2, 'name': 'Italy', 'continent': 'Europe', 'population': 59, 'gdp': 2.3}]


### Insertar datos de un CSV

In [24]:
# Creamos el CSV
import pandas as pd

df= pd.read_csv('country_data.csv', sep=';')

In [25]:
df

,name,continent,population,gdp
0,France,Europe,68,3.05
1,Japan,Asia,124,4.20
2,Argentina,America,45,0.65
3,Brazil,America,211,2.17


In [26]:
for datos in zip(*[valor.values() for clave,valor in df.to_dict().items()]):
    insert = "INSERT INTO country(name,continent,population,gdp) VALUES ('%s','%s',%d,%.1f)" \
    %(datos[0],datos[1],datos[2],datos[3])
    cursor.execute(insert)

In [35]:
query = "SELECT * FROM country_new"
cursor.execute(query)
cursor.fetchall()

[{'name': 'France', 'continent': 'Europe', 'population': 68, 'gdp': 3.05},
 {'name': 'Japan', 'continent': 'Asia', 'population': 124, 'gdp': 4.2},
 {'name': 'Argentina', 'continent': 'America', 'population': 45, 'gdp': 0.65},
 {'name': 'Brazil', 'continent': 'America', 'population': 211, 'gdp': 2.17}]

In [28]:
db.commit()

In [29]:
# Utilizando sqlalchemy
from sqlalchemy import create_engine
# import pymysql

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = "country_database"))

In [33]:
df.to_sql(name = "country_new", con= engine, if_exists= "append", index = False)

4

In [34]:
db.commit()

### Cerrar la conexión cuando acabemos

In [36]:
db.close()

In [37]:
conn = engine.connect()
conn.close()